## Feature Engineering on Diff Data
USA World Series Results,
Run on "Diff" data
Added new features for "bands" KO win % difference

In [1]:
import pandas as pd
import numpy as np

# set option to NOT truncate columns in DF display
pd.set_option('display.max_columns', None)

In [13]:
#Import Data - USA match differentials
df = pd.read_csv('../data/output/final_diffs_all_2018-19.csv')
#Import validation set from 2018-19 World Series
df.head()

,Opp,Tournament,Poss_Time_Diff,Score_Diff,Conv_Diff,Tries_Diff,Passes_Diff,Contestable_KO_Win_pct_Diff,PenFK_Against_Diff,RuckMaul_Diff,Ruck_Win_pct_Diff,Cards_diff,Lineout_Win_Pct_Diff,Scrum_Win_Pct_Diff
0,AUSTRALIA,2019_Dubai_7s,-28.601253,29.166667,25.000000,0.266667,-45.714286,-30.000000,33.333333,-100.000000,6,-50,1.0,0.0
1,Argentina,2019_Cape_Town,21.813031,22.222222,33.333333,-0.750000,28.358209,-25.000000,14.285714,100.000000,12,0,0.0,0.0
2,CANADA,2019_Sydney_7s,16.037736,44.000000,50.000000,-0.500000,19.354839,-41.666667,20.000000,100.000000,8,50,1.0,0.0
3,ENGLAND,2019_Hamilton_7s,-27.884615,46.153846,50.000000,-0.333333,-41.333333,33.333333,-33.333333,-100.000000,33,0,1.0,-1.0
4,ENGLAND,2019_Sydney_7s,14.127424,46.153846,50.000000,-0.333333,55.555556,0.000000,-50.000000,33.333333,-12,0,1.0,0.0


In [3]:
# Replace NaN's with zero
df.fillna(value=0, inplace=True)

In [4]:
# create result label: change from 1,0,2 to win, loss, tie
# if 'Score_Diff' is > 0 = win, if < 0, else tie
def createResult(x):
    if x > 0:
        return 1 #"W"
    elif x < 0:
        return 0 #"L"
    elif x == 0:
        return 2 #"T"

df.loc[:,'Result'] = df.loc[:,'Score_Diff'].apply(createResult)

In [5]:
df.head()

,Opp,Tournament,Poss_Time_Diff,Score_Diff,Conv_Diff,Tries_Diff,Passes_Diff,Contestable_KO_Win_pct_Diff,PenFK_Against_Diff,RuckMaul_Diff,Ruck_Win_pct_Diff,Cards_diff,Lineout_Win_Pct_Diff,Scrum_Win_Pct_Diff,Result
0,AUSTRALIA,2019_Dubai_7s,-28.601253,29.166667,25.000000,0.266667,-45.714286,-30.000000,33.333333,-100.000000,6,-50,1.0,0.0,1
1,Argentina,2019_Cape_Town,21.813031,22.222222,33.333333,-0.750000,28.358209,-25.000000,14.285714,100.000000,12,0,0.0,0.0,1
2,CANADA,2019_Sydney_7s,16.037736,44.000000,50.000000,-0.500000,19.354839,-41.666667,20.000000,100.000000,8,50,1.0,0.0,1
3,ENGLAND,2019_Hamilton_7s,-27.884615,46.153846,50.000000,-0.333333,-41.333333,33.333333,-33.333333,-100.000000,33,0,1.0,-1.0,1
4,ENGLAND,2019_Sydney_7s,14.127424,46.153846,50.000000,-0.333333,55.555556,0.000000,-50.000000,33.333333,-12,0,1.0,0.0,1


In [6]:
df.Contestable_KO_Win_pct_Diff.describe()

count    24.000000
mean    -12.549603
std      40.380869
min     -60.000000
25%     -44.642857
50%     -25.000000
75%       2.500000
max      75.000000
Name: Contestable_KO_Win_pct_Diff, dtype: float64

In [7]:
df.Contestable_KO_Win_pct_Diff

0    -30.000000
1    -25.000000
2    -41.666667
3     33.333333
4      0.000000
5     50.000000
6     10.000000
7     75.000000
8      0.000000
9    -50.000000
10   -42.857143
11   -50.000000
12   -20.000000
13   -50.000000
14     0.000000
15   -50.000000
16   -50.000000
17   -40.000000
18    75.000000
19   -40.000000
20   -25.000000
21    30.000000
22   -60.000000
23     0.000000
Name: Contestable_KO_Win_pct_Diff, dtype: float64

In [8]:
#Create columns for KO win % bands based on Contestable_KO_Win_pct_Diff
# Tmp DF to hold values
tmp = pd.DataFrame(columns=['Less than -75', '-50 : -75', '-25 : -49', '24 : -24', '25 : 49', '50 : 75', 'More than 75'])

#7/2/18 - revised number/range of bins
# <-75
# -50 : -75
# -25 : -49
# <25 : >-25
# 25 : 49
# 50 : 75
# >75

#Iterate through rows and create classification for KO Win%
for index, row in df.iterrows():
    
    if row['Contestable_KO_Win_pct_Diff'] < -75.0:
        bin1 = row['Less than -75'] = float(100)
        bin2 = row['-50 : -75'] = float(0)
        bin3 = row['-25 : -49'] = float(0)
        bin4 = row['24 : -24'] = float(0)
        bin5 = row['25 : 49'] = float(0)
        bin6 = row['50 : 75'] = float(0)
        bin7 = row['More than 75'] = float(0)
        
    elif row['Contestable_KO_Win_pct_Diff'] <= -50.0 and row['Contestable_KO_Win_pct_Diff'] >= -75.0:
        bin1 = row['Less than -75'] = float(0)
        bin2 = row['-50 : -75'] = float(100)
        bin3 = row['-25 : -49'] = float(0)
        bin4 = row['24 : -24'] = float(0)
        bin5 = row['25 : 49'] = float(0)
        bin6 = row['50 : 75'] = float(0)
        bin7 = row['More than 75'] = float(0)
      
    elif row['Contestable_KO_Win_pct_Diff'] <= -25.0 and row['Contestable_KO_Win_pct_Diff'] >= -49.0:
        bin1 = row['Less than -75'] = float(0)
        bin2 = row['-50 : -75'] = float(0)
        bin3 = row['-25 : -49'] = float(100)
        bin4 = row['24 : -24'] = float(0)
        bin5 = row['25 : 49'] = float(0)
        bin6 = row['50 : 75'] = float(0)
        bin7 = row['More than 75'] = float(0)        
         
    elif row['Contestable_KO_Win_pct_Diff'] <= 24.0 and row['Contestable_KO_Win_pct_Diff'] >= -24.0:
        bin1 = row['Less than -75'] = float(0)
        bin2 = row['-50 : -75'] = float(0)
        bin3 = row['-25 : -49'] = float(0)
        bin4 = row['24 : -24'] = float(100)
        bin5 = row['25 : 49'] = float(0)
        bin6 = row['50 : 75'] = float(0)
        bin7 = row['More than 75'] = float(0)       
          
    elif row['Contestable_KO_Win_pct_Diff'] >= 25.0 and row['Contestable_KO_Win_pct_Diff'] <= 49.0:
        bin1 = row['Less than -75'] = float(0)
        bin2 = row['-50 : -75'] = float(0)
        bin3 = row['-25 : -49'] = float(0)
        bin4 = row['24 : -24'] = float(0)
        bin5 = row['25 : 49'] = float(100)
        bin6 = row['50 : 75'] = float(0)
        bin7 = row['More than 75'] = float(0)       
          
    elif row['Contestable_KO_Win_pct_Diff'] >= 50.0 and row['Contestable_KO_Win_pct_Diff'] <= 75:
        bin1 = row['Less than -75'] = float(0)
        bin2 = row['-50 : -75'] = float(0)
        bin3 = row['-25 : -49'] = float(0)
        bin4 = row['24 : -24'] = float(0)
        bin5 = row['25 : 49'] = float(0)
        bin6 = row['50 : 75'] = float(100)
        bin7 = row['More than 75'] = float(0)        
        
    elif row['Contestable_KO_Win_pct_Diff'] > 75:
        bin1 = row['Less than -75'] = float(0)
        bin2 = row['-50 : -75'] = float(0)
        bin3 = row['-25 : -49'] = float(0)
        bin4 = row['24 : -24'] = float(0)
        bin5 = row['25 : 49'] = float(0)
        bin6 = row['50 : 75'] = float(0)
        bin7 = row['More than 75'] = float(100)
        
    tmp.loc[index] = (bin1, bin2, bin3, bin4, bin5, bin6, bin7)
    
#df.info()
#type(df.Result[1])
df = pd.concat([df, tmp], axis=1)

#df.to_csv("../data/output/matchdata_ko_bands.csv", header=True, index=False)

#print(list(df.columns))
# ['Team', 'Date', 'Tournament', 'Match', 'Possession Time', 'Scores', 'Tries', 'Conversions', 'Passes', 'Contestable_KO_Win_pct_Diff', 'Pens_Frees Against', 'Ruck_Maul', 'Yellow_Red Cards', 'TurnoversConceded', 'Ruck_retention', 'Lineout_Win_Pct', 'Scrum_Win_Pct', 'Result', '0', '0 - 25', '25 - 50', '50 - 75', '75 - 100']
#df.head()

#Create columns for KO win % bands based on Contestable_KO_Win_pct_Diff
#Tmp DF to hold values
tmp = pd.DataFrame(columns=['-175 : -150', '-149 : -125', '-124 : -100', '-99 : -75', '-74 : -25', '-24 : -1', '0 : 25', '26 : 50', '51 : 75', '76 : 100', '101 : 125', '126 : 150'])

#Iterate through rows and create classification for KO Win%
for index, row in df.iterrows():
    
    if row['Contestable_KO_Win_pct_Diff'] > -175.0 and row['Contestable_KO_Win_pct_Diff'] <= -150.0:
        bin0 = row['-175 : -150'] = float(-1.50*50)
        bin1 = row['-149 : -125'] = float(0)
        bin2 = row['-124 : -100'] = float(0)
        bin3 = row['-99 : -75'] = float(0)
        bin4 = row['-74 : -25'] = float(0)
        bin5 = row['-24 : -1'] = float(0)
        bin6 = row['0 : 25'] = float(0)
        bin7 = row['26 : 50'] = float(0)
        bin8 = row['51 : 75'] = float(0)
        bin9 = row['76 : 100'] = float(0)
        bin10 = row['101 : 125'] = float(0)
        bin11 = row['126 : 150'] = float(0)
        
    elif row['Contestable_KO_Win_pct_Diff'] > -150.0 and row['Contestable_KO_Win_pct_Diff'] <= -125.0:
        bin0 = row['-175 : -150'] = float(0)
        bin1 = row['-149 : -125'] = float(-1.25*50)
        bin2 = row['-124 : -100'] = float(0)
        bin3 = row['-99 : -75'] = float(0)
        bin4 = row['-74 : -25'] = float(0)
        bin5 = row['-24 : -1'] = float(0)
        bin6 = row['0 : 25'] = float(0)
        bin7 = row['26 : 50'] = float(0)
        bin8 = row['51 : 75'] = float(0)
        bin9 = row['76 : 100'] = float(0)
        bin10 = row['101 : 125'] = float(0)
        bin11 = row['126 : 150'] = float(0)  
      
    elif row['Contestable_KO_Win_pct_Diff'] > -125.0 and row['Contestable_KO_Win_pct_Diff'] <= -100.0:
        bin0 = row['-175 : -150'] = float(0)
        bin1 = row['-149 : -125'] = float(0)
        bin2 = row['-124 : -100'] = float(-1.00*50)
        bin3 = row['-99 : -75'] = float(0)
        bin4 = row['-74 : -25'] = float(0)
        bin5 = row['-24 : -1'] = float(0)
        bin6 = row['0 : 25'] = float(0)
        bin7 = row['26 : 50'] = float(0)
        bin8 = row['51 : 75'] = float(0)
        bin9 = row['76 : 100'] = float(0)
        bin10 = row['101 : 125'] = float(0)
        bin11 = row['126 : 150'] = float(0)         
         
    elif row['Contestable_KO_Win_pct_Diff'] > -100.0 and row['Contestable_KO_Win_pct_Diff'] <= -75.0:
        bin0 = row['-175 : -150'] = float(0)
        bin1 = row['-149 : -125'] = float(0)
        bin2 = row['-124 : -100'] = float(0)
        bin3 = row['-99 : -75'] = float(-0.75*50)
        bin4 = row['-74 : -25'] = float(0)
        bin5 = row['-24 : -1'] = float(0)
        bin6 = row['0 : 25'] = float(0)
        bin7 = row['26 : 50'] = float(0)
        bin8 = row['51 : 75'] = float(0)
        bin9 = row['76 : 100'] = float(0)
        bin10 = row['101 : 125'] = float(0)
        bin11 = row['126 : 150'] = float(0)        
          
    elif row['Contestable_KO_Win_pct_Diff'] > -75.0 and row['Contestable_KO_Win_pct_Diff'] <= -25.0:
        bin0 = row['-175 : -150'] = float(0)
        bin1 = row['-149 : -125'] = float(0)
        bin2 = row['-124 : -100'] = float(0)
        bin3 = row['-99 : -75'] = float(0)
        bin4 = row['-74 : -25'] = float(-0.25*50)
        bin5 = row['-24 : -1'] = float(0)
        bin6 = row['0 : 25'] = float(0)
        bin7 = row['26 : 50'] = float(0)
        bin8 = row['51 : 75'] = float(0)
        bin9 = row['76 : 100'] = float(0)
        bin10 = row['101 : 125'] = float(0)
        bin11 = row['126 : 150'] = float(0)        
          
    elif row['Contestable_KO_Win_pct_Diff'] > -25.0 and row['Contestable_KO_Win_pct_Diff'] <= -1.0:
        bin0 = row['-175 : -150'] = float(0)
        bin1 = row['-149 : -125'] = float(0)
        bin2 = row['-124 : -100'] = float(0)
        bin3 = row['-99 : -75'] = float(0)
        bin4 = row['-74 : -25'] = float(0)
        bin5 = row['-24 : -1'] = float(-0.10*50)
        bin6 = row['0 : 25'] = float(0)
        bin7 = row['26 : 50'] = float(0)
        bin8 = row['51 : 75'] = float(0)
        bin9 = row['76 : 100'] = float(0)
        bin10 = row['101 : 125'] = float(0)
        bin11 = row['126 : 150'] = float(0)          
        
    elif row['Contestable_KO_Win_pct_Diff'] >= 0 and row['Contestable_KO_Win_pct_Diff'] <= 25.0:
        bin0 = row['-175 : -150'] = float(0)
        bin1 = row['-149 : -125'] = float(0)
        bin2 = row['-124 : -100'] = float(0)
        bin3 = row['-99 : -75'] = float(0)
        bin4 = row['-74 : -25'] = float(0)
        bin5 = row['-24 : -1'] = float(0)
        bin6 = row['0 : 25'] = float(0.25*50)
        bin7 = row['26 : 50'] = float(0)
        bin8 = row['51 : 75'] = float(0)
        bin9 = row['76 : 100'] = float(0)
        bin10 = row['101 : 125'] = float(0)
        bin11 = row['126 : 150'] = float(0)

    elif row['Contestable_KO_Win_pct_Diff'] > 25.0 and row['Contestable_KO_Win_pct_Diff'] <= 50.0:
        bin0 = row['-175 : -150'] = float(0)
        bin1 = row['-149 : -125'] = float(0)
        bin2 = row['-124 : -100'] = float(0)
        bin3 = row['-99 : -75'] = float(0)
        bin4 = row['-74 : -25'] = float(0)
        bin5 = row['-24 : -1'] = float(0)
        bin6 = row['0 : 25'] = float(0)
        bin7 = row['26 : 50'] = float(0.50*50)
        bin8 = row['51 : 75'] = float(0)
        bin9 = row['76 : 100'] = float(0)
        bin10 = row['101 : 125'] = float(0)
        bin11 = row['126 : 150'] = float(0)

    elif row['Contestable_KO_Win_pct_Diff'] > 50.0 and row['Contestable_KO_Win_pct_Diff'] <= 75.0:
        bin0 = row['-175 : -150'] = float(0)
        bin1 = row['-149 : -125'] = float(0)
        bin2 = row['-124 : -100'] = float(0)
        bin3 = row['-99 : -75'] = float(0)
        bin4 = row['-74 : -25'] = float(0)
        bin5 = row['-24 : -1'] = float(0)
        bin6 = row['0 : 25'] = float(0)
        bin7 = row['26 : 50'] = float(0)
        bin8 = row['51 : 75'] = float(0.75*50)
        bin9 = row['76 : 100'] = float(0)
        bin10 = row['101 : 125'] = float(0)
        bin11 = row['126 : 150'] = float(0)

    elif row['Contestable_KO_Win_pct_Diff'] > 75.0 and row['Contestable_KO_Win_pct_Diff'] <= 100.0:
        #zero = row['0'] = float(0)
        bin0 = row['-175 : -150'] = float(0)
        bin1 = row['-149 : -125'] = float(0)
        bin2 = row['-124 : -100'] = float(0)
        bin3 = row['-99 : -75'] = float(0)
        bin4 = row['-74 : -25'] = float(0)
        bin5 = row['-24 : -1'] = float(0)
        bin6 = row['0 : 25'] = float(0)
        bin7 = row['26 : 50'] = float(0)
        bin8 = row['51 : 75'] = float(0)
        bin9 = row['76 : 100'] = float(1.00*50)
        bin10 = row['101 : 125'] = float(0)
        bin11 = row['126 : 150'] = float(0)

    elif row['Contestable_KO_Win_pct_Diff'] > 100.0 and row['Contestable_KO_Win_pct_Diff'] <= 125.0:
        bin0 = row['-175 : -150'] = float(0)
        bin1 = row['-149 : -125'] = float(0)
        bin2 = row['-124 : -100'] = float(0)
        bin3 = row['-99 : -75'] = float(0)
        bin4 = row['-74 : -25'] = float(0)
        bin5 = row['-24 : -1'] = float(0)
        bin6 = row['0 : 25'] = float(0)
        bin7 = row['26 : 50'] = float(0)
        bin8 = row['51 : 75'] = float(0)
        bin9 = row['76 : 100'] = float(0)
        bin10 = row['101 : 125'] = float(1.25*50)
        bin11 = row['126 : 150'] = float(0)        
        

    elif row['Contestable_KO_Win_pct_Diff'] > 125.0 and row['Contestable_KO_Win_pct_Diff'] <= 150.0:
        #zero = row['0'] = float(0)
        bin0 = row['-175 : -150'] = float(0)
        bin1 = row['-149 : -125'] = float(0)
        bin2 = row['-124 : -100'] = float(0)
        bin3 = row['-99 : -75'] = float(0)
        bin4 = row['-74 : -25'] = float(0)
        bin5 = row['-24 : -1'] = float(0)
        bin6 = row['0 - 25'] = float(0)
        bin7 = row['26 - 50'] = float(0)
        bin8 = row['51 - 75'] = float(0)
        bin9 = row['76 - 100'] = float(0)
        bin10 = row['101 - 125'] = float(0)
        bin11 = row['126 - 150'] = float(1.50*50) 
        
    tmp.loc[index] = (bin0, bin1, bin2, bin3, bin4, bin5, bin6, bin7, bin8, bin9, bin10, bin11)
    
#df.info()
#type(df.Result[1])
df = pd.concat([df, tmp], axis=1)

In [9]:
#df.info()
list(df.columns)

['Opp',
 'Tournament',
 'Poss_Time_Diff',
 'Score_Diff',
 'Conv_Diff',
 'Tries_Diff',
 'Passes_Diff',
 'Contestable_KO_Win_pct_Diff',
 'PenFK_Against_Diff',
 'RuckMaul_Diff',
 'Ruck_Win_pct_Diff',
 'Cards_diff',
 'Lineout_Win_Pct_Diff',
 'Scrum_Win_Pct_Diff',
 'Result',
 'Less than -75',
 '-50 : -75',
 '-25 : -49',
 '24 : -24',
 '25 : 49',
 '50 : 75',
 'More than 75']

In [10]:
# reorder columns
df = df[['Opp','Tournament','Poss_Time_Diff','Score_Diff','Conv_Diff','Tries_Diff','Passes_Diff','Contestable_KO_Win_pct_Diff','PenFK_Against_Diff','RuckMaul_Diff','Ruck_Win_pct_Diff','Cards_diff','Lineout_Win_Pct_Diff','Scrum_Win_Pct_Diff','Less than -75','-50 : -75','-25 : -49','24 : -24','25 : 49','50 : 75','More than 75','Result']]
df.head()

,Opp,Tournament,Poss_Time_Diff,Score_Diff,Conv_Diff,Tries_Diff,Passes_Diff,Contestable_KO_Win_pct_Diff,PenFK_Against_Diff,RuckMaul_Diff,Ruck_Win_pct_Diff,Cards_diff,Lineout_Win_Pct_Diff,Scrum_Win_Pct_Diff,Less than -75,-50 : -75,-25 : -49,24 : -24,25 : 49,50 : 75,More than 75,Result
0,AUSTRALIA,2019_Dubai_7s,-28.601253,29.166667,25.000000,0.266667,-45.714286,-30.000000,33.333333,-100.000000,6,-50,1.0,0.0,0.0,0.0,100.0,0.0,0.0,0.0,0.0,1
1,Argentina,2019_Cape_Town,21.813031,22.222222,33.333333,-0.750000,28.358209,-25.000000,14.285714,100.000000,12,0,0.0,0.0,0.0,0.0,100.0,0.0,0.0,0.0,0.0,1
2,CANADA,2019_Sydney_7s,16.037736,44.000000,50.000000,-0.500000,19.354839,-41.666667,20.000000,100.000000,8,50,1.0,0.0,0.0,0.0,100.0,0.0,0.0,0.0,0.0,1
3,ENGLAND,2019_Hamilton_7s,-27.884615,46.153846,50.000000,-0.333333,-41.333333,33.333333,-33.333333,-100.000000,33,0,1.0,-1.0,0.0,0.0,0.0,0.0,100.0,0.0,0.0,1
4,ENGLAND,2019_Sydney_7s,14.127424,46.153846,50.000000,-0.333333,55.555556,0.000000,-50.000000,33.333333,-12,0,1.0,0.0,0.0,0.0,0.0,100.0,0.0,0.0,0.0,1


In [11]:
#bins = df[['Less than -75','-50 : -75','-25 : -49','24 : -24','25 : 49','50 : 75','More than 75'].describe()
#print(bins)

In [12]:
df.to_csv("../data/output/new_features_diffdata_validate_2018-19.csv", header=True, index=False)
#Create validation set
#df.to_csv("../data/output/new_features_diffdata_validate.csv", header=True, index=False)